### Imports

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import tensorflow as tf
import mysql.connector
import plotly.graph_objects as go
import os
import xgboost as xgb
import plotly.figure_factory as ff
import dash
from sklearn.preprocessing import MinMaxScaler
from dash import Dash, dcc, html, Input, Output 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score

### DB Connection

In [2]:
user = os.environ.get("DB_USER")
password = os.environ.get("DB_PASS")

In [3]:
connection = mysql.connector.connect(host = 'localhost',
                                    user = user,
                                    passwd = password,
                                    db = 'school_shootings')

In [4]:
# Entire Dataset
shootings = pd.read_sql_query('''
SELECT
    * 
FROM
    `school-shootings-data`
''', connection)
shootings.head()

,uid,nces_school_id,school_name,nces_district_id,district_name,date,school_year,year,time,day_of_week,...,low_grade,high_grade,lunch,county,state_fips,county_fips,ulocale,party,gfi,state_pop
0,1,80480000707,Columbine High School,804800,Jefferson County R-1,4/20/1999,1998-1999,1999,11:19 AM,Tuesday,...,9,12,41,Jefferson County,8,8059,21,dem,39,5961083
1,2,220000000000,Scotlandville Middle School,2200540,East Baton Rouge Parish School Board,4/22/1999,1998-1999,1999,12:30 PM,Thursday,...,6,8,495,East Baton Rouge Parish,22,22033,12,rep,34,4616106
2,3,130000000000,Heritage High School,1304410,Rockdale County,5/20/1999,1998-1999,1999,8:03 AM,Thursday,...,9,12,125,Rockdale County,13,13247,21,dem,23,10830007
3,4,422000000000,John Bartram High School,4218990,Philadelphia City SD,10/4/1999,1999-2000,1999,10:00 AM,Monday,...,9,12,2007,Philadelphia County,42,42101,11,dem,28,12804123
4,5,250000000000,Dorchester High School,2502790,Boston,11/3/1999,1999-2000,1999,7:40 AM,Wednesday,...,9,12,543,Suffolk County,25,25025,11,dem,49,6922107


### EDA

In [5]:
# Some questions I want to anwser:
# What state has had the most school shootings?
# Is there a relation to a states political stance and the amount of school shootings that occur?
# Is there a relation to the severity of gun laws in a state and the amount of school shootings that occur?
# Is there a relation between school size and shootings that occur?
# Is there a relation between state population and the amount of school shootings that occur?
# Have school shootings been increasing in frequency since 1998? 

In [35]:
# What state has had the most school shootings?
state_with_most_shootings_figure = px.histogram(shootings,
             x='state',
             title='School Shootings by State',
             labels={"state": "State"}).update_xaxes(categoryorder='total descending')
state_with_most_shootings_figure.show()

In [36]:
# Is there a relation to a states political stance and the amount of school shootings that occur?
political_stance_shootings = px.histogram(
             shootings,
             x='state',
             color='party',
             title='School Shootings by State Color-coded by Political Party',
             labels={"count": "Number of Shootings","state": "State"}
            ).update_xaxes(categoryorder='total descending')
political_stance_shootings.show()

In [37]:
# Lets make a query to get the number of shootings per state and the states "gun friendly index" number
# This number comes from https://www.az-defenders.com/best-states-for-gun-owners/
# Assumptions being made: the more gun friendly a state, the looser the laws.
# 1 means the most gun friendly, 51 means the least (dc is counted)

shooting_count = pd.read_sql_query('''

SELECT 
    state,
    COUNT(state) as NumberOfShootings,
    gfi,
    state_pop
from `school-shootings-data`
GROUP BY state
''', connection)
shooting_count.head()

,state,NumberOfShootings,gfi,state_pop
0,Colorado,7,39,5961083
1,Louisiana,9,34,4616106
2,Georgia,10,23,10830007
3,Pennsylvania,1,28,12804123
4,Massachusetts,1,49,6922107


In [38]:
# Is there a relation to the severity of gun laws in a state and the amount of school shootings that occur?
shootings_by_gfi = px.scatter(
    shooting_count,
    x='gfi',
    y='NumberOfShootings',
    text="state",
    title='Number of School Shootings in Relation to Gun Friendly Index',
    labels={"gfi": "Gun Friendly Index",
            "NumberOfShootings": "Number of Shootings"}
)

shootings_by_gfi.update_traces(textposition='top center')
shootings_by_gfi.show()

In [39]:
# lets see if there is a relation between the amount of school shootings and state population
state_pop_shootings = px.scatter(
    shooting_count,
    x='state_pop',
    y='NumberOfShootings',
    title='Number of School Shootings by State Population',
    text='state',
    labels={"state_pop": "State Population",
            "NumberOfShootings": "Number of Shootings"}
)
state_pop_shootings.update_traces(textposition='top center')
state_pop_shootings.show()

In [40]:
# It seems like we have been hearing about school shootings more than ever these days
# Have they been increasing in frequency since 1998? 
# I will have to make a new sql query for to find the count of each shooting by year
shootings_by_year = pd.read_sql_query('''

SELECT 
    school_year,
    COUNT(state) as NumberOfShootings
from 
    `school-shootings-data`
GROUP BY 
    school_year
    
''', connection)
shootings_by_year.head()

,school_year,NumberOfShootings
0,1998-1999,3
1,1999-2000,10
2,2000-2001,11
3,2001-2002,4
4,2002-2003,8


In [41]:
# Lets see the amount of school shootings by school year
shootings_year = px.line(
    shootings_by_year,
    x='school_year',
    y='NumberOfShootings',
    title='Amount of School Shootings by School Year',
    markers=True,
    labels={"school_year": "School Year",
            "NumberOfShootings": "Number of Shootings"}
)
shootings_year.show()

In [ ]:
# What on earth happened this year?